In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import Model

# ---------------------
# Step 1: Load and Preprocess Images
# ---------------------

base_path = r'D:\Rice-Disease-Simulation\data\train'
img_size = (224, 224)

images = []
labels = []
class_names = sorted(os.listdir(base_path))
label_map = {name: idx for idx, name in enumerate(class_names)}

for class_name in class_names:
    class_path = os.path.join(base_path, class_name)
    if not os.path.isdir(class_path):
        continue
    for img_file in os.listdir(class_path):
        img_path = os.path.join(class_path, img_file)
        img = cv2.imread(img_path)
        if img is None:
            continue
        img = cv2.resize(img, img_size)
        images.append(img)
        labels.append(label_map[class_name])

images = np.array(images)
labels = np.array(labels)
images = preprocess_input(images)

print(f"✅ Loaded {len(images)} images across {len(class_names)} classes.")

# ---------------------
# Step 2: Feature Extraction using MobileNetV2
# ---------------------

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model = Model(inputs=base_model.input, outputs=base_model.output)

features = model.predict(images, batch_size=32, verbose=1)
features = features.reshape(features.shape[0], -1)

print(f"✅ Feature shape: {features.shape}")

# ---------------------
# Step 3: Train/Test Split
# ---------------------

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42, stratify=labels)

# ---------------------
# Step 4: Train Logistic Regression
# ---------------------

lr = LogisticRegression(max_iter=1000, solver='saga', multi_class='multinomial', n_jobs=-1)
lr.fit(X_train, y_train)

# ---------------------
# Step 5: Evaluate
# ---------------------

y_pred = lr.predict(X_test)

print("\n✅ Classification Report:\n")
print(classification_report(y_test, y_pred, target_names=class_names))

print("✅ Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


✅ Loaded 2100 images across 6 classes.
66/66 ━━━━━━━━━━━━━━━━━━━━ 47s 677ms/step
✅ Feature shape: (2100, 62720)


C:\Users\uzmap\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


KeyboardInterrupt: 

untouched validation folder

In [ ]:
val_path = r'D:\Rice-Disease-Simulation\data\validation'

def load_and_extract_features(folder_path, label_map, model, img_size=(224, 224)):
    val_images = []
    val_labels = []

    for class_name in sorted(os.listdir(folder_path)):
        class_folder = os.path.join(folder_path, class_name)
        if not os.path.isdir(class_folder):
            continue
        for img_file in os.listdir(class_folder):
            img_path = os.path.join(class_folder, img_file)
            img = cv2.imread(img_path)
            if img is None:
                continue
            img = cv2.resize(img, img_size)
            img = preprocess_input(img)
            val_images.append(img)
            val_labels.append(label_map.get(class_name, -1))  # Use -1 for unknown classes

    val_images = np.array(val_images, dtype=np.float32)
    val_labels = np.array(val_labels)

    print(f"\n✅ Loaded {len(val_images)} validation images.")

    val_features = model.predict(val_images, batch_size=32, verbose=1)
    val_features = val_features.reshape(val_features.shape[0], -1)

    return val_features, val_labels

# Load validation data
X_val, y_val = load_and_extract_features(val_path, label_map, model)

# Remove samples with unknown class (-1)
mask = y_val != -1
X_val = X_val[mask]
y_val = y_val[mask]

# Predict
y_val_pred = lr.predict(X_val)

# Evaluate
print("\n✅ Validation Set Classification Report:\n")
print(classification_report(y_val, y_val_pred, target_names=class_names))

print("✅ Validation Confusion Matrix:")
print(confusion_matrix(y_val, y_val_pred))